## Load libraries and setup environment

In [ ]:
# Import libraries
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
# authenticate
auth.authenticate_user()

In [ ]:
# Set up environment variables
project_id = 'my-project'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

In [ ]:
cohort = run_query("""

select
    icu.subject_id,
    icu.hadm_id,
    icu.stay_id,
    icu.gender,
    icu.admission_age,
    icu.los_icu,
    icu.dod,
    AVG(bc.wbc) AS avg_wbc_derived,
    AVG(wbc_chartevents.avg_wbc_icu) AS avg_wbc_icu,
    charlson.charlson_comorbidity_index,
    aps.apsiii,
    sofa_d1.SOFA,
    (CASE WHEN AVG(wt.weight_admit) <= 300 THEN (AVG(wt.weight_admit) / ((0.01*AVG(ht.height)) * (0.01*AVG(ht.height)))) ELSE NULL END) AS bmi_calculated,
    omr.bmi_icu AS bmi_icu,
    AVG(glucose_chartevents.mean_glucose) AS avg_mean_glucose_icu,
    AVG(glucose_chartevents.glucose_variability) AS avg_glucose_variability_icu,
    AVG(gc.mean_glucose_derived) AS mean_glucose_derived,
    AVG(gc.glucose_variability_derived) AS glucose_variability_derived

from physionet-data.mimiciv_derived.icustay_detail as icu

--defines the sepsis population
inner join physionet-data.mimiciv_derived.sepsis3 as s3
on s3.stay_id = icu.stay_id

--study population filter
and icu.first_icu_stay is true
and icu.first_hosp_stay is true # first stay condition
AND icu.los_icu >= 1.0  # at least one day in icu condition
AND icu.admission_age >= 18 # age over 18 condition
and s3.sepsis3 is true

--Charlson
left join physionet-data.mimiciv_derived.charlson as charlson
on icu.hadm_id = charlson.hadm_id

--WBC
LEFT JOIN `physionet-data.mimiciv_derived.complete_blood_count` AS bc ON icu.subject_id = bc.subject_id AND icu.hadm_id = bc.hadm_id

LEFT JOIN (
  SELECT ce.subject_id, ce.hadm_id, ce.stay_id, AVG(LEAST(ce.valuenum, 300)) AS avg_wbc_icu
  FROM `physionet-data.mimiciv_icu.chartevents` AS ce
  JOIN (
    SELECT label, itemid
    FROM `physionet-data.mimiciv_icu.d_items`
    WHERE label LIKE '%WBC%'
  ) AS di ON ce.itemid = di.itemid
  GROUP BY ce.subject_id, ce.hadm_id, ce.stay_id
) AS wbc_chartevents
ON icu.subject_id = wbc_chartevents.subject_id AND icu.hadm_id = wbc_chartevents.hadm_id AND icu.stay_id = wbc_chartevents.stay_id

--weight and height
left join physionet-data.mimiciv_derived.first_day_height as ht
on icu.stay_id = ht.stay_id --ensure only first stay
left join physionet-data.mimiciv_derived.first_day_weight as wt
on icu.stay_id = wt.stay_id

-- bmi
LEFT JOIN (
    SELECT o.subject_id, o.chartdate, CAST(o.result_value AS FLOAT64) AS bmi_icu
    FROM `physionet-data.mimiciv_hosp.omr` AS o
    WHERE result_name LIKE '%BMI%' AND seq_num = 1
    GROUP BY o.subject_id, o.chartdate, bmi_icu
  ) AS omr ON omr.subject_id = icu.subject_id AND omr.chartdate = icu.admittime

--APACHE III
left join physionet-data.mimiciv_derived.apsiii as aps
on icu.stay_id = aps.stay_id

--SOFA
left join physionet-data.mimiciv_derived.first_day_sofa as sofa_d1
on icu.stay_id = sofa_d1.stay_id

-- glucose
INNER JOIN (
  SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.stay_id,
    AVG(LEAST(ce.valuenum, 500)) AS mean_glucose,
    STDDEV(LEAST(ce.valuenum, 500)) AS glucose_variability
  FROM `physionet-data.mimiciv_icu.chartevents` AS ce
  JOIN (
    SELECT label, itemid
    FROM `physionet-data.mimiciv_icu.d_items`
    WHERE label = 'Glucose (serum)' OR label = 'Glucose (whole blood)'
  ) AS di ON ce.itemid = di.itemid
  JOIN (
    SELECT icu.subject_id, icu.hadm_id, icu.intime,
           DATETIME_ADD(icu.intime, INTERVAL 24 HOUR) AS BG_reading_time_limit
    FROM `physionet-data.mimiciv_icu.icustays` AS icu
  ) AS admission_times ON ce.subject_id = admission_times.subject_id
  WHERE ce.charttime >= admission_times.intime AND ce.charttime <= admission_times.BG_reading_time_limit
  GROUP BY ce.subject_id, ce.hadm_id, ce.stay_id
) AS glucose_chartevents ON icu.subject_id = glucose_chartevents.subject_id AND icu.hadm_id = glucose_chartevents.hadm_id AND icu.stay_id = glucose_chartevents.stay_id

LEFT JOIN (
  SELECT
    ce.subject_id,
    ce.hadm_id,
    AVG(LEAST(ce.glucose, 500)) AS mean_glucose_derived,
    STDDEV(LEAST(ce.glucose, 500)) AS glucose_variability_derived
  FROM `physionet-data.mimiciv_derived.bg` AS ce
  JOIN (
    SELECT
      icu.subject_id,
      icu.hadm_id,
      icu.admittime,
      DATETIME_ADD(icu.admittime, INTERVAL 24 HOUR) AS BG_reading_time_limit
      FROM `physionet-data.mimiciv_derived.icustay_detail` AS icu
  ) AS admission_times ON ce.subject_id = admission_times.subject_id
  WHERE ce.charttime >= admission_times.admittime AND ce.charttime <= admission_times.BG_reading_time_limit
  GROUP BY ce.subject_id, ce.hadm_id
) AS gc ON icu.subject_id = gc.subject_id AND icu.hadm_id = gc.hadm_id

GROUP BY
icu.subject_id,
icu.hadm_id,
icu.stay_id,
icu.gender,
icu.admission_age,
icu.los_icu,
icu.hospital_expire_flag,
icu.dod,
charlson.charlson_comorbidity_index,
aps.apsiii,
sofa_d1.SOFA,
omr.bmi_icu

""")

In [ ]:
cohort.info()

In [ ]:
selected_cohort = cohort[['subject_id', 'hadm_id', 'stay_id', 'admission_age', 'gender', 'bmi_calculated', 'bmi_icu', 'SOFA', 'apsiii', 'charlson_comorbidity_index', 'los_icu', 'dod',
'avg_wbc_derived','avg_wbc_icu','avg_mean_glucose_icu','mean_glucose_derived','avg_glucose_variability_icu','glucose_variability_derived']]

In [ ]:
selected_cohort

In [ ]:
selected_cohort_summary = selected_cohort.describe()
selected_cohort_summary

In [ ]:
max_value = selected_cohort['bmi_calculated'].max()
print("Maximum bmi_calculated value:", max_value)
max_value = selected_cohort['bmi_icu'].max()
print("Maximum bmi_icu value:", max_value)

In [ ]:
cohort_with_null = selected_cohort[selected_cohort['avg_wbc_derived'].isnull()]
print(len(cohort_with_null))
cohort_with_null = selected_cohort[selected_cohort['bmi_calculated'].isnull()]
print(len(cohort_with_null))
cohort_with_null = selected_cohort[selected_cohort['avg_mean_glucose_icu'].isnull()]
print(len(cohort_with_null))
cohort_with_null = selected_cohort[selected_cohort['avg_glucose_variability_icu'].isnull()]
print(len(cohort_with_null))


In [ ]:
df = selected_cohort.copy()
df['avg_wbc_derived'].fillna(df['avg_wbc_icu'], inplace=True)
df['bmi_calculated'].fillna(df['bmi_icu'], inplace=True)
df['avg_mean_glucose_icu'].fillna(df['mean_glucose_derived'], inplace=True)
df['avg_glucose_variability_icu'].fillna(df['glucose_variability_derived'], inplace=True)
df['dod'] = df['dod'].notnull().astype(int)

In [ ]:
df_with_null = df[df['avg_wbc_derived'].isnull()]
print(len(df_with_null))
df_with_null = df[df['bmi_calculated'].isnull()]
print(len(df_with_null))
df_with_null = df[df['avg_mean_glucose_icu'].isnull()]
print(len(df_with_null))
df_with_null = df[df['avg_glucose_variability_icu'].isnull()]
print(len(df_with_null))
dod = (df['dod'] == 1).sum()
print("Number of records where dod is 1:", dod)

In [ ]:
print(df.columns)

In [ ]:
columns_to_drop = ['subject_id', 'hadm_id', 'stay_id', 'bmi_icu', 'avg_wbc_icu', 'mean_glucose_derived', 'glucose_variability_derived']

df_dropped = df.drop(columns=columns_to_drop)

In [ ]:
df_dropped

In [ ]:
column_name_mapping = {
    'admission_age': 'age',
    'gender': 'gender',
    'bmi_calculated': 'bmi',
    'SOFA': 'SOFA',
    'apsiii': 'apsiii',
    'charlson_comorbidity_index': 'charlson_comorbidity_index',
    'los_icu': 'los',
    'dod': 'survival_status',
    'avg_wbc_derived': 'wbc',
    'avg_mean_glucose_icu': 'blood_glucose',
    'avg_glucose_variability_icu': 'glucose_variability',
}

# Use the rename method to rename the columns
df_dropped.rename(columns=column_name_mapping, inplace=True)

In [ ]:
df_dropped


In [ ]:
df_dropped.to_csv("final_cohort_dod.csv", index=False)